<H1>Machine Learning Projesi NLP KULLANARAK IMDB FİLM YORUMLARI </H1>


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.feature_extraction.text import CountVectorizer#NLP için kullanılır
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from bs4 import BeautifulSoup
import re
import nltk
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords

In [3]:

df=pd.read_csv('NLPlabeledData.tsv',delimiter='\t',quoting=3)

In [4]:
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [5]:
len(df)

25000

In [6]:
len(df["review"])

25000

In [8]:
nltk.download('stopwords')
#stopwords'ü temizlemek için önce indirmemiz gerekiyor.

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tugce\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

<h1>veri temizleme</h1>
öncelikle beautifulSoup modulunu kullanarak HTML taglerini reviewden silinip
tek bir review seçip nasıl yapıldığına bakılır.


In [9]:
sample_review=df.review[0]
sample_review

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [10]:
#HTML tagleri temizlenmesi
sample_review=BeautifulSoup(sample_review).get_text()
sample_review

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 2

In [12]:
#noktalama işaretleerinden ve sayılardan temizliyoruz regex kullanarak
sample_review=re.sub("[^a-zA-Z]",' ',sample_review)
sample_review

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    m

In [13]:
#tüm metni küçük harfe dönüştürüyoruz makinenin büyük harfli kelimeleri başka algılamaması için
sample_review=sample_review.lower()
sample_review

' with all this stuff going down at the moment with mj i ve started listening to his music  watching the odd documentary here and there  watched the wiz and watched moonwalker again  maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  some of it has subtle messages about mj s feeling towards the press and also the obvious message of drugs are bad m kay visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring  some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him the actual feature film bit when it finally starts is only on for    m

In [14]:
#stopwords kelimelerini önce split ediyoruz kelimeleri bölüyoruz bir listeye dönüştürüyoruz amacımız bunları çıakrmak 
sample_review=sample_review.split()
sample_review

['with',
 'all',
 'this',
 'stuff',
 'going',
 'down',
 'at',
 'the',
 'moment',
 'with',
 'mj',
 'i',
 've',
 'started',
 'listening',
 'to',
 'his',
 'music',
 'watching',
 'the',
 'odd',
 'documentary',
 'here',
 'and',
 'there',
 'watched',
 'the',
 'wiz',
 'and',
 'watched',
 'moonwalker',
 'again',
 'maybe',
 'i',
 'just',
 'want',
 'to',
 'get',
 'a',
 'certain',
 'insight',
 'into',
 'this',
 'guy',
 'who',
 'i',
 'thought',
 'was',
 'really',
 'cool',
 'in',
 'the',
 'eighties',
 'just',
 'to',
 'maybe',
 'make',
 'up',
 'my',
 'mind',
 'whether',
 'he',
 'is',
 'guilty',
 'or',
 'innocent',
 'moonwalker',
 'is',
 'part',
 'biography',
 'part',
 'feature',
 'film',
 'which',
 'i',
 'remember',
 'going',
 'to',
 'see',
 'at',
 'the',
 'cinema',
 'when',
 'it',
 'was',
 'originally',
 'released',
 'some',
 'of',
 'it',
 'has',
 'subtle',
 'messages',
 'about',
 'mj',
 's',
 'feeling',
 'towards',
 'the',
 'press',
 'and',
 'also',
 'the',
 'obvious',
 'message',
 'of',
 'drugs',

In [15]:
len(sample_review)

437

In [16]:
#silme işlemi
swords=set(stopwords.words("english"))
sample_review=[w for w in sample_review if w not in swords]
sample_review

['stuff',
 'going',
 'moment',
 'mj',
 'started',
 'listening',
 'music',
 'watching',
 'odd',
 'documentary',
 'watched',
 'wiz',
 'watched',
 'moonwalker',
 'maybe',
 'want',
 'get',
 'certain',
 'insight',
 'guy',
 'thought',
 'really',
 'cool',
 'eighties',
 'maybe',
 'make',
 'mind',
 'whether',
 'guilty',
 'innocent',
 'moonwalker',
 'part',
 'biography',
 'part',
 'feature',
 'film',
 'remember',
 'going',
 'see',
 'cinema',
 'originally',
 'released',
 'subtle',
 'messages',
 'mj',
 'feeling',
 'towards',
 'press',
 'also',
 'obvious',
 'message',
 'drugs',
 'bad',
 'kay',
 'visually',
 'impressive',
 'course',
 'michael',
 'jackson',
 'unless',
 'remotely',
 'like',
 'mj',
 'anyway',
 'going',
 'hate',
 'find',
 'boring',
 'may',
 'call',
 'mj',
 'egotist',
 'consenting',
 'making',
 'movie',
 'mj',
 'fans',
 'would',
 'say',
 'made',
 'fans',
 'true',
 'really',
 'nice',
 'actual',
 'feature',
 'film',
 'bit',
 'finally',
 'starts',
 'minutes',
 'excluding',
 'smooth',
 'crim

In [17]:
len(sample_review)

219

In [18]:
#Temizleme işlemini tüm reviewler için yapacağız bunun için bir fonksiyon oluşturuyoruz


In [19]:
def process(review):
    #review without HTML tags
    review=BeautifulSoup(review).get_text()
    #review without punctuation and numbers
    review=re.sub("[^a-zA-Z]",' ',review)
    #converting into lowercase and splitting to eliminate stopwords
    review=review.lower()
    review=review.split()
    #review without stopwords
    swords=set(stopwords.words("english"))
    review=[w for w in review if w not in swords]
    #splitted paragraf'ları space ile birleştiriyoruz
    return(" ".join(review))
    
    

In [23]:
#tarinig datamızı yukarıda temizliyoruz her 1000 review sonrası durumu kontrol ediyoruz
train_x_tum=[]
for r in range(len(df["review"])):
    if(r+1)%1000==0:
        print("No of reviews processed",r+1)
    train_x_tum.append(process(df["review"][r]))


C:\Users\tugce\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


No of reviews processed 1000
No of reviews processed 2000
No of reviews processed 3000
No of reviews processed 4000
No of reviews processed 5000
No of reviews processed 6000
No of reviews processed 7000
No of reviews processed 8000
No of reviews processed 9000
No of reviews processed 10000
No of reviews processed 11000
No of reviews processed 12000
No of reviews processed 13000
No of reviews processed 14000
No of reviews processed 15000
No of reviews processed 16000
No of reviews processed 17000
No of reviews processed 18000
No of reviews processed 19000
No of reviews processed 20000
No of reviews processed 21000
No of reviews processed 22000
No of reviews processed 23000
No of reviews processed 24000
No of reviews processed 25000


### train tespit


In [25]:
x=train_x_tum
y=np.array(df["sentiment"])
train_x,test_x,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=42)

# bag of words oluşturuyoruz 
verilerimizi temizledik ancak yapay zekanın  anlaması için bu metin tabanlı verileri sayılara ve bag of words denilen matrise çevirmelyiz.

In [27]:
#sklearn içinde bulunan countvectorizer fonksiyonunu kullanarak max 5000 kelimelik bag of words oluşturuyoruz
vectorizer=CountVectorizer(max_features=5000)
#train verilerimizi feature vectorüne matrise çeviriyoruz
train_x=vectorizer.fit_transform(train_x)
train_x

<22500x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 1776690 stored elements in Compressed Sparse Row format>

In [28]:
#bunu array'e dönüştürüyoruz fit edebilmek için 
train_x=train_x.toarray()

In [29]:
train_y=y_train
train_x.shape,train_y.shape

((22500, 5000), (22500,))

In [30]:
train_y#yorumları tuttar 1 se olumlu bir yorum 

array([0, 1, 1, ..., 1, 1, 0], dtype=int64)

Random forest modeli oluturuyoruz ve fit ediyoruz


In [31]:
model = RandomForestClassifier(n_estimators=100)
model.fit(train_x,train_y)

RandomForestClassifier()

# Tesst datamız


In [32]:
#test verilerimizi feature vektöre matrisine çeviriyoruz
#yani aynı işlemleri bag of word işlemlerini tekrarlıyoruz bu sefr test datamız için 
test_xx=vectorizer.transform(test_x)
test_xx

<2500x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 198330 stored elements in Compressed Sparse Row format>

In [33]:
test_xx=test_xx.toarray()

In [35]:
test_xx.shape

(2500, 5000)

prediction yapıyoruz


In [36]:
test_predict = model.predict(test_xx)
dogruluk = roc_auc_score(y_test,test_predict)

In [37]:
print("doğruluk oranı:%",dogruluk*100)

doğruluk oranı:% 83.84485178128777
